In [2]:
import sys
import dotenv
import os
dotenv.load_dotenv()
sys.path.append(os.getenv('MAINDIR'))
from helpers.ds_helpers import X_y_split, open_and_prepare_df
from helpers.models_helpers import random_seeds
import tensorflow as tf
import numpy as np
from scipy import stats
import warnings
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error


warnings.filterwarnings("ignore", category=UserWarning)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

X, y = X_y_split(open_and_prepare_df('features'), 'nlp_all')

2023-04-25 12:08:34.209445: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-25 12:08:34.231880: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 12:08:34.534776: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-25 12:08:34.948041: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

Num GPUs Available:  1


Manual parameters search

In [9]:
def create_model():
    random_seeds()
    keras_model = tf.keras.Sequential([
                        tf.keras.layers.Dense(300, activation='sigmoid'),
                        tf.keras.layers.Dense(150, activation='LeakyReLU'),
                        tf.keras.layers.Dense(64, activation='LeakyReLU'),
                        
                        tf.keras.layers.Dense(units=1, activation='selu')
                        ])

    keras_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.85), loss='mean_absolute_error')
    return keras_model

In [10]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=8)

keras_predictions = []
y_main_true = []

for train_index, test_index in kfold.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = create_model()
    model.fit(X_train, y_train, batch_size=70, epochs=150, verbose=0)
    keras_prediction = model.predict(X_test, verbose=0).flatten()

    keras_predictions = np.concatenate((keras_predictions, keras_prediction))
    y_main_true = np.concatenate((y_main_true, y_test))
    print(stats.pearsonr(keras_prediction, y_test))
    
stats.pearsonr(y_main_true, keras_predictions)

PearsonRResult(statistic=0.5602176557319587, pvalue=7.669735201215567e-05)
PearsonRResult(statistic=0.6554783945823128, pvalue=1.373531985306179e-06)
PearsonRResult(statistic=0.5812894026681479, pvalue=3.508713499212738e-05)
PearsonRResult(statistic=0.36080204328407395, pvalue=0.017458892669358)
PearsonRResult(statistic=0.514659182224191, pvalue=0.00041414177161451965)
PearsonRResult(statistic=0.42304774383387567, pvalue=0.004707227231263233)
PearsonRResult(statistic=0.5987923361546916, pvalue=2.2175598643418047e-05)
PearsonRResult(statistic=0.5171110963129416, pvalue=0.0003843052635653643)
PearsonRResult(statistic=0.545634018546047, pvalue=0.0001542408029089122)
PearsonRResult(statistic=0.42852437195781995, pvalue=0.004144392099843738)


PearsonRResult(statistic=0.5066653860605428, pvalue=1.292724354015643e-29)